<a href="https://colab.research.google.com/github/WayneGretzky1/CSCI-4521-Applied-Machine-Learning/blob/main/Copy_of_2_2_text_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Download and extract the data
Data is in a zip file

In [ ]:
!wget "https://raw.githubusercontent.com/be-prado/csci4521/refs/heads/main/20news-bydate.tar.gz"

--2025-09-29 19:44:18--  https://raw.githubusercontent.com/be-prado/csci4521/refs/heads/main/20news-bydate.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14464277 (14M) [application/octet-stream]
Saving to: ‘20news-bydate.tar.gz’

20news-bydate.tar.g 100%[===================>]  13.79M  --.-KB/s    in 0.05s   

2025-09-29 19:44:18 (253 MB/s) - ‘20news-bydate.tar.gz’ saved [14464277/14464277]



In [ ]:
!tar -xf 20news-bydate.tar.gz

## Using SciKit-Learn's CountVectorizer


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

The parameter `min_df` controls effect words that are not used frequently (min_df = minimum document frequency).
 - If it is an integer, all words occurring less than that value will be dropped.
 - If it is a fraction, all words that occur less than that fraction of the overall dataset are be dropped.

`max_df` works in a similar manner

In [ ]:
vectorizer = CountVectorizer(min_df=1) #min_df=1 --> use all words

In [ ]:
CountVectorizer?

Consider two sentences:

In [ ]:
content = ["How to catch pokemon", "Which Pokemon is the hardest to catch?"]

How many uniuqe words between the two?
  - Is `catch` and `catch?` the same word?
  - Is `Pokemon` and `pokemon` the same word?
  - Would `catch` and `catching` be the same word?

In [ ]:
# TODO: fit_transform the sentences then print the vocab
X = vectorizer.fit_transform(content)

vectorizer.get_feature_names_out()



array(['catch', 'hardest', 'how', 'is', 'pokemon', 'the', 'to', 'which'],
      dtype=object)

We can turn each sentence into a "bag of words" ... for each sentence:
 - 1 is word is present
 - 0 is word is absent

In [ ]:
print(X.toarray())

[[1 0 1 0 1 0 1 0]
 [1 1 0 1 1 1 1 1]]


### CountVectorizer on UseNet posts

In [ ]:
import os
DIR = "/content/20news-bydate-train/rec.sport.hockey"

In [ ]:
posts = [open(os.path.join(DIR, filename)).read() for filename in os.listdir(DIR)]

In [ ]:
posts[50]

"From: wdsst3@cislabs.pitt.edu (William D Sands)\nSubject: request for video in Pittsburgh area\nKeywords: Sunday afternoon\nOrganization: University of Pittsburgh\nLines: 11\n\n\n\tThere was apparently a 30 minute special here on the Penguins' \nseason on ABC (WTAE - channel 4), immediately preceding the opening \ngame against the Devils on Sunday.  I only turned it on in time to \nwatch the credits.  If anyone taped it and is willing to let me borrow \nit to dub it, I would appreciate it.  I would be willing to come pick \nit up, and I'll return it the next day and buy you a beer.  Please \nrespond via e-mail.  Thanks a lot.\n\tOh yeah.  Was it any good?\n\t\t\t\t\t\t-Billy\n\n"

In [ ]:
# TODO: fit_transform the vectorizer with our new data
X_train = vectorizer.fit_transform(posts)

In [ ]:
len(posts)

600

In [ ]:
X_train.shape

(600, 12914)

In [ ]:
print(vectorizer.get_feature_names_out())
print(len(vectorizer.get_feature_names_out()))

['00' '000' '000256' ... 'zupancic' 'zurich' 'zzzzzz']
12914


In [ ]:
# TODO: vectorize the sentence "Should a team be added in Wisconsin?"
new_post = "Should a team be added in Wisconsin?"
new_post_vec = vectorizer.transform([new_post])
print(new_post_vec)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (1, 12914)>
  Coords	Values
  (0, 1900)	1
  (0, 2610)	1
  (0, 6536)	1
  (0, 10730)	1
  (0, 11577)	1
  (0, 12665)	1


In [ ]:
# TODO: get the names of features present
vectorizer.get_feature_names_out()[1900]
vectorizer.get_feature_names_out()[2610]
vectorizer.get_feature_names_out()[6536]
vectorizer.get_feature_names_out()[10730]
vectorizer.get_feature_names_out()[11577]
vectorizer.get_feature_names_out()[12665]

'wisconsin'

### Finding Nearest Neighbors

`new_post_vec` is a feature vector, and we can try to find its nearest neighbors in the training set

In [ ]:
import numpy as np

In [ ]:
def dist_raw(v1, v2):
  delta = v1-v2
  return np.linalg.norm(delta)

In [ ]:
# TODO: find the distances between the new post and the vectors in our training set
dists = [dist_raw(new_post_vec.toarray(), train_vec) for train_vec in X_train]

In [ ]:
# TODO: which post is the closest?
best_match_ID = np.argmin(dists)
print(best_match_ID)

159


In [ ]:
dists[best_match_ID]

np.float64(5.291502622129181)

In [ ]:
posts[best_match_ID]

'From: khettry@r1w2.pub.utk.edu (23064RFL)\nSubject: Testing!!!\nOrganization: University of Tennessee Computing Center\n\tJust Testing!!\nDistribution: usa\nLines: 1\n\n\n'

Hmm. The querry document was `"Should a new team be added to Wisconsin?"`.

Does this post seem related to our query feature? Let's check which elements of the feature vectors overlap.

In [ ]:
# TODO: print the query vector and the closest vector?
print(X_train[best_match_ID])


<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 19 stored elements and shape (1, 12914)>
  Coords	Values
  (0, 5633)	1
  (0, 11333)	1
  (0, 8805)	1
  (0, 12149)	1
  (0, 8685)	1
  (0, 7497)	1
  (0, 4851)	1
  (0, 6966)	1
  (0, 3371)	1
  (0, 4579)	1
  (0, 12209)	1
  (0, 3789)	1
  (0, 7085)	1
  (0, 9693)	1
  (0, 9607)	1
  (0, 12230)	1
  (0, 917)	1
  (0, 11651)	2
  (0, 11623)	1


In [ ]:
print(new_post_vec)

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6 stored elements and shape (1, 12914)>
  Coords	Values
  (0, 1900)	1
  (0, 2610)	1
  (0, 6536)	1
  (0, 10730)	1
  (0, 11577)	1
  (0, 12665)	1


That worked poorly... There is no overlap in features. What happened?

#### Normalized distance
Normalizing vectors before computing distance focuses on document content rather than length

In [ ]:
def dist_norm(v1, v2):
  v1_normalized = v1/np.linalg.norm(v1) #Normalize vectors to unit length
  v2_normalized = v2/np.linalg.norm(v2)
  delta = v1_normalized-v2_normalized   #Then take distance
  return np.linalg.norm(delta)

In [ ]:
# TODO: find the normalized distances between the new post and the vectors in our training set then find the new closest post
dists = [dist_norm(new_post_vec.toarray(), train_vec.toarray()) for train_vec in X_train]

In [ ]:
best_match_ID = np.argmin(dists)
print(best_match_ID)

467


In [ ]:
posts[best_match_ID]

"From: jake@rambler.Eng.Sun.COM (Jason Cockroft)\nSubject: Re: Playoff predictions\nOrganization: Sun Microsystems, Inc.\nLines: 95\nDistribution: world\nReply-To: jake@rambler.Eng.Sun.COM\nNNTP-Posting-Host: rambler.eng.sun.com\n\n1st round: \n----------\n\nPITT vs NYI:  PITT in 4. \n\nIt looks like a safe bet.  NYI has been bagging it of late.\nNYI and NJD have a showdown Friday night for the honour\nof Pittsburg anyway.  Pigsburg in 4.\n\nWASH vs NJD:  WASH in 6. \nI think that NJD have a solid team and will compete with\nWASH.  I agree though with WASH in 6.\n\nBOS  vs BUF:  BOS  in 5. \nThe B's have been playing awesome hockey in the last\ntwo weeks.  The only question is how long will it last?\nFuhr is a dud.  BOS in 4.\n\nQUE  vs MON:  MON  in 7.\nIt seems to me that MON is much like the VAN - no chemistry.\nThe Habs seem to be not in stride. QUE in 5.  \n\nCHI  vs STL:  CHI in 4. \nSTL should not be in the playoffs. CHI in 4.\n\nDET  vs TOR:  DET in 6.\nI am a diehard Leaf fan 

## Stop Words, Stemming, and TF-IDF
Ignoring common words (stop words)

In [ ]:
vectorizer = CountVectorizer(min_df=1, stop_words='english')

We'll lose some words now. The size of the feature vector should be smaller.

In [ ]:
X_train.shape #Old Vectorizations

(600, 12914)

In [ ]:
X_train = vectorizer.fit_transform(posts)

In [ ]:
X_train.shape #New Vectorizations

(600, 12633)

In [ ]:
sorted(vectorizer.get_stop_words())

['a',
 'about',
 'above',
 'across',
 'after',
 'afterwards',
 'again',
 'against',
 'all',
 'almost',
 'alone',
 'along',
 'already',
 'also',
 'although',
 'always',
 'am',
 'among',
 'amongst',
 'amoungst',
 'amount',
 'an',
 'and',
 'another',
 'any',
 'anyhow',
 'anyone',
 'anything',
 'anyway',
 'anywhere',
 'are',
 'around',
 'as',
 'at',
 'back',
 'be',
 'became',
 'because',
 'become',
 'becomes',
 'becoming',
 'been',
 'before',
 'beforehand',
 'behind',
 'being',
 'below',
 'beside',
 'besides',
 'between',
 'beyond',
 'bill',
 'both',
 'bottom',
 'but',
 'by',
 'call',
 'can',
 'cannot',
 'cant',
 'co',
 'con',
 'could',
 'couldnt',
 'cry',
 'de',
 'describe',
 'detail',
 'do',
 'done',
 'down',
 'due',
 'during',
 'each',
 'eg',
 'eight',
 'either',
 'eleven',
 'else',
 'elsewhere',
 'empty',
 'enough',
 'etc',
 'even',
 'ever',
 'every',
 'everyone',
 'everything',
 'everywhere',
 'except',
 'few',
 'fifteen',
 'fifty',
 'fill',
 'find',
 'fire',
 'first',
 'five',
 'for'

In [ ]:
# TODO: based on a new query post, which post in our dataset is closest?
new_post = "Would I get injured playing goalie?"
new_post_vec = vectorizer.transform([new_post])
dists = [dist_norm(new_post_vec.toarray(), train_vec.toarray()) for train_vec in X_train]
best_match_ID = np.argmin(dists)

print(best_match_ID)
print(posts[best_match_ID])

273
From: drozinst@db.erau.edu (Drozinski Tim)
Subject: Re: Ulf and all...
Organization: Embry-Riddle Aeronautical University, Daytona Beach, FL USA
Lines: 59
NNTP-Posting-Host: alpha.db.erau.edu

layfield@cpsc.ucalgary.ca (Colin Layfield) writes:

>In article <1pdlksINNmq7@GIRAFFE.ZOO.CS.YALE.EDU> wuziyun%suned@cs.yale.edu (You wanna know?) writes:
>>
>>
>>Let me give my two cents worth in this whole thing:
>>
>>  I am very sick of Pittsburg fans(and they are my second favorite team) 
>>talk about how " why can't Bruins forget about Ulf Samuelsson when we have
>>forgotten all about Adam Graves"  Beside the obvious fact that Lemeuix's career
>>was never endangered by Graves' slash while Neely is still bother by his injury,
>>I think the most important reason is:
>>
>>        ADAM GRAVES HAS PLAYED CLEAN HOCKEY EVER SINCE!  WHILE ULF SAMULESSON
>>        CONTINUES TO PLAY DIRTY(YES, TRYING TO HIT A PLAYER WHERE HE'S INJURED
>>        IS DIRTY).  FANS HAVE CAN FORGET ABOUT ONE DIRTY PLAY

How did this do?

We can can also add stemming and tf-idf:

In [ ]:
import nltk.stem
english_stemmer = nltk.stem.SnowballStemmer('english')

In [ ]:
english_stemmer.stem("happily")

'happili'

In [ ]:
class StemmedCountVectorizer(CountVectorizer):
   def build_analyzer(self):
     analyzer = super(StemmedCountVectorizer, self).build_analyzer()
     return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [ ]:
vectorizer = StemmedCountVectorizer(min_df=1, stop_words='english')
X_train = vectorizer.fit_transform(posts)

In [ ]:
# TODO: based on a new query post, which post in our dataset is closest?
new_post = "Would I get injured playing goalie?"
new_post_vec = vectorizer.transform([new_post])
dists = [dist_norm(new_post_vec.toarray(), train_vec.toarray()) for train_vec in X_train]
best_match_ID = np.argmin(dists)

print(best_match_ID)
print(posts[best_match_ID])

245
From: apland@mala.bc.ca (Ron Apland)
Subject: Re: plus minus stat
Organization: Malaspina College
Lines: 28

In article <1993Apr15.161737.28377@sol.UVic.CA>, gballent@hudson.UVic.CA (Greg  Ballentine) writes:
> 
> In article 1@tnclus.tele.nokia.fi, hahietanen@tnclus.tele.nokia.fi () writes:
>>In article <1993Apr14.174139.6604@sol.UVic.CA>, gballent@vancouver.UVic.CA (Greg  Ballentine) writes:
>>> 
>>> 
>>> +/- is a good stat because it is the only stat that I am aware of that
>>> takes into account defensive play.  It isn't a measure of defensive
>>> play- it takes into account offense and defence- all aspects of play.
>>                                                   
>>  If we are interested of real all-round players, the power play stats
>>  should be considered, too. Because the power play is also one aspect 
>>  of play! There is still something to be done with these player evaluation
>>  tools!!
> 
> IMO any good player should score on power plays because of the man
> adva

In [ ]:
X_train.shape

(600, 10381)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
class StemmedTfidfVectorizer(TfidfVectorizer):
  def build_analyzer(self):
    analyzer = super(TfidfVectorizer, self).build_analyzer()
    return lambda doc: (english_stemmer.stem(w) for w in analyzer(doc))

In [ ]:
vectorizer = StemmedTfidfVectorizer(min_df=1, stop_words='english')
X_train = vectorizer.fit_transform(posts)

In [ ]:
# TODO: based on a new query post, which post in our dataset is closest?
new_post = "Would I get injured playing goalie?"
new_post_vec = vectorizer.transform([new_post])
dists = [dist_norm(new_post_vec.toarray(), train_vec.toarray()) for train_vec in X_train]
best_match_ID = np.argmin(dists)

print(best_match_ID)
print(posts[best_match_ID])

273
From: drozinst@db.erau.edu (Drozinski Tim)
Subject: Re: Ulf and all...
Organization: Embry-Riddle Aeronautical University, Daytona Beach, FL USA
Lines: 59
NNTP-Posting-Host: alpha.db.erau.edu

layfield@cpsc.ucalgary.ca (Colin Layfield) writes:

>In article <1pdlksINNmq7@GIRAFFE.ZOO.CS.YALE.EDU> wuziyun%suned@cs.yale.edu (You wanna know?) writes:
>>
>>
>>Let me give my two cents worth in this whole thing:
>>
>>  I am very sick of Pittsburg fans(and they are my second favorite team) 
>>talk about how " why can't Bruins forget about Ulf Samuelsson when we have
>>forgotten all about Adam Graves"  Beside the obvious fact that Lemeuix's career
>>was never endangered by Graves' slash while Neely is still bother by his injury,
>>I think the most important reason is:
>>
>>        ADAM GRAVES HAS PLAYED CLEAN HOCKEY EVER SINCE!  WHILE ULF SAMULESSON
>>        CONTINUES TO PLAY DIRTY(YES, TRYING TO HIT A PLAYER WHERE HE'S INJURED
>>        IS DIRTY).  FANS HAVE CAN FORGET ABOUT ONE DIRTY PLAY

In [ ]:
X_train.shape

In [ ]:
# TODO: with these new vectorizers, lets test that query post again


#### Cosine Similarity

We can use the cosine similarity instead of the normalized vector distance.

But remember to maximize similarity vs. minimize distance.

In [ ]:
def cos_similarity(v1, v2):
  return np.vdot(v1,v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [ ]:
# TODO: use cosine similarity as a distance metric and try the query post again
# TODO: based on a new query post, which post in our dataset is closest?
new_post = "Would I get injured playing goalie?"
new_post_vec = vectorizer.transform([new_post])
dists = [dist_norm(new_post_vec.toarray(), train_vec.toarray()) for train_vec in X_train]
best_match_ID = np.argmax(dists)

print(best_match_ID)
print(posts[best_match_ID])

278
From: andrew@idacom.hp.com (Andrew Scott)
Subject: USENET Hockey Draft week 27 price list
Organization: Hewlett-Packard, IDACOM Telecommunications Division
Lines: 264

Here is the price list for the week April 13 to April 19.

	- Andrew

Buy	Sell	Pts	Team	Player
158.9	143.0	157	PIT	Mario_Lemieux
148.5	133.7	145	BUF	Pat_LaFontaine
142.7	128.4	141	BOS	Adam_Oates
137.6	123.8	136	DET	Steve_Yzerman
132.1	118.9	129	WPG	Teemu_Selanne
131.7	118.5	127	NYI	Pierre_Turgeon
130.1	117.1	127	TOR	Doug_Gilmour
126.0	113.4	123	BUF	Alexander_Mogilny
123.4	111.1	119	PHI	Mark_Recchi
121.9	109.7	119	LA	Luc_Robitaille
113.3	102.0	112	QUE	Mats_Sundin
111.3	100.2	110	PIT	Kevin_Stevens
110.6	99.5	108	VAN	Pavel_Bure
108.6	97.7	106	STL	Craig_Janney
108.3	97.5	107	PIT	Rick_Tocchet
107.6	96.8	105	CHI	Jeremy_Roenick
105.3	94.8	104	QUE	Joe_Sakic
103.5	93.2	101	STL	Brett_Hull
102.4	92.2	100	CGY	Theoren_Fleury
101.2	91.1	100	PIT	Ron_Francis
100.4	90.4	98	TOR	Dave_Andreychuk
100.2	90.2	99	BOS	Joe_Juneau
98.3	88.5	96

## Closest Document Function

In [ ]:
# Helper function!
def findClosestStory(promt):
  new_post_vec = vectorizer.transform([promt])
  dists = [cos_similarity(new_post_vec.toarray(),train_vec.toarray()) for train_vec in X_train]
  closest_id = np.argmax(dists) #switch to arg max!
  return posts[closest_id]


In [ ]:
print(findClosestStory("What was the name of the player that scored for the Minnesota wild?"))

From: hamlet@stein.u.washington.edu (Mitch McGowan)
Subject: Minnesota recalls McGowan (HELP!!!)
Organization: University of Washington
Lines: 24
NNTP-Posting-Host: stein.u.washington.edu

  Derian Hatcher's game-misconduct penalty was rescinded by the NHL, 
allowing the Minnesota defenseman to play in the North Stars' last two 
regular-season games.  Hatcher was given the penalty during a fight at 
the end of a loss at St. Louis on Sunday, April 11.  But the league 
didn't rescind the game-misconduct penalty Shane Churla received.  The 
Stars recalled center Cal McGowan from their top minor league club in 
Kalamazoo, Mich., to replace Churla.

The above is courtesy of The Washington Times on-line service.

Now, here's where I need help.  If anyone out there has a tape of Tuesday's
Chicago-Minnesota game, please contact me.  Terms will be favorable.

Also, if anyone can tape tonight's Minnesota-Detroit game, please contact
me.  This could be quite important.  Once again, I will make it